In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [76]:
df = pd.read_csv("./AAPL_last_40_days.csv")
df=df[:40]
df

,Date,Close,High,Low,Open
0,2025/4/4,188.133301,199.618240,187.094654,193.636079
1,2025/4/7,181.222366,193.895735,174.391312,176.967936
2,2025/4/8,172.194199,190.090729,168.988411,186.455496
3,2025/4/9,198.589584,200.347274,171.664885,171.724805
4,2025/4/10,190.170624,194.524914,182.760343,188.822401
5,2025/4/11,197.890488,199.278667,185.816325,185.856281
6,2025/4/14,202.254776,212.661128,200.896557,211.163092
7,2025/4/15,201.875275,203.243476,199.538343,201.595643
8,2025/4/16,194.015579,200.437151,192.118058,198.100219
9,2025/4/17,196.722031,198.569614,194.165386,196.941744


In [77]:
tensor_data = torch.tensor(df[['Close', 'High', 'Low', 'Open']].values, dtype=torch.float32)
print(tensor_data.shape)

torch.Size([40, 4])


In [4]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # 建立一個位置編碼矩陣
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶數位置
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇數位置
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x 的形狀為 (seq_length, batch_size, d_model)
        x = x + self.pe[:x.size(0)]
        return x
class TransformerTimeSeriesAutoregressive(nn.Module):
    def __init__(self, feature_size, num_layers=2, nhead=4, hidden_dim=128, lstm_hidden_dim=128, dropout=0.3):
        super(TransformerTimeSeriesAutoregressive, self).__init__()
        self.model_type = 'Autoregressive Transformer with LSTM Decoder and Teacher Forcing'

        self.input_linear = nn.Linear(feature_size, hidden_dim)
        self.pos_encoder = PositionalEncoding(d_model=hidden_dim)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dim_feedforward=hidden_dim, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Linear projection for h_0 and c_0
        self.h0_projection = nn.Linear(hidden_dim, lstm_hidden_dim)
        self.c0_projection = nn.Linear(hidden_dim, lstm_hidden_dim)

        self.lstm_decoder = nn.LSTM(input_size=hidden_dim, hidden_size=lstm_hidden_dim, num_layers=1, batch_first=False)
        self.output_linear = nn.Linear(lstm_hidden_dim, feature_size)

        self.residual_projection = nn.Linear(feature_size, hidden_dim) if feature_size != hidden_dim else None
        self.hidden_dim = hidden_dim
        self.lstm_hidden_dim = lstm_hidden_dim

    def forward(self, src, target=None, teacher_forcing_ratio=0.5):
        batch_size, seq_length, feature_size = src.size()
        pred_length = target.size(1) if target is not None else 5

        if self.residual_projection:
            residual_src = self.residual_projection(src)
        else:
            residual_src = src
        
        residual_src = residual_src.permute(1, 0, 2)
        src = src.permute(1, 0, 2)
        src = self.input_linear(src)
        src = self.pos_encoder(src)

        transformer_output = self.transformer_encoder(src)
        transformer_output = transformer_output + residual_src

        # Use the last timestep's output to initialize h_0 and c_0
        last_step_output = transformer_output[-1]  # (batch_size, hidden_dim)
        h_0 = self.h0_projection(last_step_output).unsqueeze(0)  # (1, batch_size, lstm_hidden_dim)
        c_0 = self.c0_projection(last_step_output).unsqueeze(0)  # (1, batch_size, lstm_hidden_dim)

        # Start autoregressive decoding
        decoder_input = transformer_output[-1, :, :].unsqueeze(0)  # (1, batch_size, hidden_dim)
        outputs = []

        for t in range(pred_length):
            lstm_output, (h_0, c_0) = self.lstm_decoder(decoder_input, (h_0, c_0))
            lstm_output = lstm_output + decoder_input
            output = self.output_linear(lstm_output.squeeze(0))
            outputs.append(output.unsqueeze(1))

            if target is not None and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input = self.input_linear(target[:, t, :]).unsqueeze(0)
            else:
                decoder_input = self.input_linear(output).unsqueeze(0)

        outputs = torch.cat(outputs, dim=1)
        return outputs


In [5]:
# 檢查是否有可用的GPU
if torch.cuda.is_available():
    print("GPU")
    device = torch.device("cuda")  # 使用GPU
else:
    print("CPU")
    device = torch.device("cpu")  # 使用CPU

GPU


In [78]:
x = tensor_data


# 最小值與最大值沿著 dim=0（即每欄）
data_min = x.min(dim=0, keepdim=True).values
data_max = x.max(dim=0, keepdim=True).values

# 避免除以 0：加一個極小值 epsilon
epsilon = 1e-8
normalized_x = (x - data_min) / (data_max - data_min + epsilon)

print(normalized_x.shape)  # 應為 torch.Size([40, 4])

torch.Size([40, 4])


In [79]:
model = torch.load('./5day/trans_lstm_5.pth')
model.to(device)
y = torch.zeros((5, 4), dtype=torch.float32)
normalized_x = torch.unsqueeze(normalized_x,0)
y = torch.unsqueeze(y,0)
pred = model(normalized_x.to(device), y.to(device), teacher_forcing_ratio=0).squeeze(-1)

In [80]:
print(pred[0][0][0])

tensor(0.6430, device='cuda:0', grad_fn=<SelectBackward0>)


In [81]:
pred[0][0][0] * (data_max[0][0] - data_min[0][0]) + data_min[0][0]

tensor(198.4585, device='cuda:0', grad_fn=<AddBackward0>)

# 輸入真實值

In [74]:
newdf = df.iloc[1:].reset_index(drop=True)
# 建立一筆新資料（用實際值取代這裡的例子）
new_row = pd.DataFrame([{
    'Date':'2025/6/2',
    'Close': 201.70,
    'High': 202.13,
    'Low': 200.12,
    'Open': 200.28
}])

# 插入到最後一行
newdf = pd.concat([newdf, new_row], ignore_index=True)
newdf

,Date,Close,High,Low,Open
0,2025/4/4,188.133301,199.618240,187.094654,193.636079
1,2025/4/7,181.222366,193.895735,174.391312,176.967936
2,2025/4/8,172.194199,190.090729,168.988411,186.455496
3,2025/4/9,198.589584,200.347274,171.664885,171.724805
4,2025/4/10,190.170624,194.524914,182.760343,188.822401
5,2025/4/11,197.890488,199.278667,185.816325,185.856281
6,2025/4/14,202.254776,212.661128,200.896557,211.163092
7,2025/4/15,201.875275,203.243476,199.538343,201.595643
8,2025/4/16,194.015579,200.437151,192.118058,198.100219
9,2025/4/17,196.722031,198.569614,194.165386,196.941744


In [75]:
newdf.to_csv("AAPL_last_40_days.csv", index=False)